In [2]:
import json
import pandas as pd
import numpy as np
import re
import random
import sqlite3

with open('lookup1.json') as json_file: 
    data = json.load(json_file)

with open('uniquelookup.json') as json_file:
    data2 = json.load(json_file)



In [3]:
conn = sqlite3.connect('testQ.db')
c = conn.cursor()


In [5]:
question_template = 'What is the number of (Bed Entity) in (Location Entity)?'
state_dict = {'AK': 'Alaska',
        'AL': 'Alabama',
        'AR': 'Arkansas',
        'AS': 'American Samoa',
        'AZ': 'Arizona',
        'CA': 'California',
        'CO': 'Colorado',
        'CT': 'Connecticut',
        'DC': 'District of Columbia',
        'DE': 'Delaware',
        'FL': 'Florida',
        'GA': 'Georgia',
        'GU': 'Guam',
        'HI': 'Hawaii',
        'IA': 'Iowa',
        'ID': 'Idaho',
        'IL': 'Illinois',
        'IN': 'Indiana',
        'KS': 'Kansas',
        'KY': 'Kentucky',
        'LA': 'Louisiana',
        'MA': 'Massachusetts',
        'MD': 'Maryland',
        'ME': 'Maine',
        'MI': 'Michigan',
        'MN': 'Minnesota',
        'MO': 'Missouri',
        'MP': 'Northern Mariana Islands',
        'MS': 'Mississippi',
        'MT': 'Montana',
        'NA': 'National',
        'NC': 'North Carolina',
        'ND': 'North Dakota',
        'NE': 'Nebraska',
        'NH': 'New Hampshire',
        'NJ': 'New Jersey',
        'NM': 'New Mexico',
        'NV': 'Nevada',
        'NY': 'New York',
        'OH': 'Ohio',
        'OK': 'Oklahoma',
        'OR': 'Oregon',
        'PA': 'Pennsylvania',
        'PR': 'Puerto Rico',
        'RI': 'Rhode Island',
        'SC': 'South Carolina',
        'SD': 'South Dakota',
        'TN': 'Tennessee',
        'TX': 'Texas',
        'UT': 'Utah',
        'VA': 'Virginia',
        'VI': 'Virgin Islands',
        'VT': 'Vermont',
        'WA': 'Washington',
        'WI': 'Wisconsin',
        'WV': 'West Virginia',
        'WY': 'Wyoming'}
output = {}
question_template_id = 'db3q1'
count = 1
print(data['Location Entity'])
question_key = {}
def replaceBedState(name):
    query = ""
    if bed == 'staffed beds':
        query = sql_template.replace("NUM_Licensed/Staffed/ICU_Beds", "NUM_STAFFED_BEDS")
        query = query.replace("STATE_Name/COUNTY_Name = input", "STATE_NAME = '" + name + "'")
    elif bed == 'licensed beds':
        query = sql_template.replace("NUM_Licensed/Staffed/ICU_Beds", "NUM_LICENSED_BEDS")
        query = query.replace("STATE_Name/COUNTY_Name = input", "STATE_NAME = '" + name + "'")
    elif bed == 'ICU beds':
        query = sql_template.replace("NUM_Licensed/Staffed/ICU_Beds", "NUM_ICU_BEDS")
        query = query.replace("STATE_Name/COUNTY_Name = input", "STATE_NAME = '" + name + "'")
    return query

def replaceBedCounty(c_name, s_name):
    query = ""
    if bed == 'staffed beds':
        query = sql_template.replace("NUM_Licensed/Staffed/ICU_Beds", "NUM_STAFFED_BEDS")
        query = query.replace("STATE_Name/COUNTY_Name = input", "COUNTY_NAME = '" + c_name + "'")
        query = query + " AND STATE_NAME = " + "'" + s_name + "'"
    elif bed == 'licensed beds':
        query = sql_template.replace("NUM_Licensed/Staffed/ICU_Beds", "NUM_LICENSED_BEDS")
        query = query.replace("STATE_Name/COUNTY_Name = input", "COUNTY_NAME = '" + c_name + "'")
        query = query + " AND STATE_NAME = " + "'" + s_name + "'"
    elif bed == 'ICU beds':
        query = sql_template.replace("NUM_Licensed/Staffed/ICU_Beds", "NUM_ICU_BEDS")
        query = query.replace("STATE_Name/COUNTY_Name = input", "COUNTY_NAME = '" + c_name + "'")
        query = query + " AND STATE_NAME = " + "'" + s_name + "'"
    return query
while count<100:  
    output[count] = []
    bed = random.choice(data['Bed Entity'])
    location = random.choice(data['Location Entity'])
    while location.find('Country') >= 0 or location.find('Province') >= 0 or location.find('(') < 0: 
        location = random.choice(data['Location Entity'])
    if location == 'in the state of (State)': 
        location = 'the state of (State)'
    entity = re.findall('\(([^)]+)', location)
    query = ""
    populated_entities = []
    sql_template = 'Select Sum(NUM_Licensed/Staffed/ICU_Beds) From db3 Where STATE_Name/COUNTY_Name = input'
    if len(entity) == 1:  
              if entity[0] == 'State': 
                  real_question = question_template.replace("(Bed Entity)", bed)
                  state_name = random.choice(data2['State'])
                  loc = location.replace("(State)", state_name)
                  real_question = real_question.replace("(Location Entity)", loc)
                  query = replaceBedState(state_name)
                  populated_entities.append(bed)
                  populated_entities.append(state_name)
                 
              if entity[0] == 'State Abbreviation': 
                  real_question = question_template.replace("(Bed Entity)", bed)
                  state_abbreviation = random.choice(data2['State Abbreviation'])
                  loc = location.replace("(State Abbreviation)", state_abbreviation)
                  real_question = real_question.replace("(Location Entity)", loc)
                  query = replaceBedState(state_dict[state_abbreviation])
                  populated_entities.append(bed)
                  populated_entities.append(state_abbreviation)
    else: 
        if entity[1] =='State':
            real_question = question_template.replace("(Bed Entity)", bed)
            county_list = random.choice(data2['County']).split(", ")
            county_name = county_list[0]
            state_name = county_list[1]
            loc = location.replace("(County)", county_name)
            loc = loc.replace("(State)", state_name)
            real_question = real_question.replace("(Location Entity)", loc)
            query = replaceBedCounty(county_name, state_name)
            populated_entities.append(bed)
            populated_entities.append(county_name + ', ' + state_name)
        if entity[1] == 'State Abbreviation': 
            real_question = question_template.replace("(Bed Entity)", bed)
            county_list = random.choice(data2['County']).split(", ")
            county_name = county_list[0]
            state_name = county_list[1]
            key_list = list(state_dict.keys())
            val_list = list(state_dict.values())
            state_abbreviation = key_list[val_list.index(state_name)]
            loc = location.replace("(County)", county_name)
            loc = loc.replace("(State Abbreviation)", state_abbreviation)
            real_question = real_question.replace("(Location Entity)", loc)
            query = replaceBedCounty(county_name, state_name)
            populated_entities.append(bed)
            populated_entities.append(county_name + ', ' + state_abbreviation)
    c.execute(query)
    result = c.fetchall()
    print(real_question)
    
    if result[0][0] == None: 
        continue
    elif real_question in question_key.keys():
        continue
    else:
        question_key[real_question] = True
        output[count].append({'question_template_id' : question_template_id, 'question_template' : question_template, 
        'entities' : ['Bed Entity', 'Location Entity'], 'question' : real_question, 
        'populated_entities': populated_entities, 'query_template' : sql_template, 'query' :  query, 'database': 'database 3'})
        count = count + 1
print(output)
  
                
                



['(Province)', '(Province), (Country)', '(Province) in (Country)', '(County), (State)', '(County) in (State)', 'in the state of (State)', '(County), (State Abbreviation)', '(State)', '(State Abbreviation)', '(State), (Country)', '(Country)', 'the country of (Country)', 'in the world/around the world', 'globally']
What is the number of licensed beds in Washington, IL?
What is the number of licensed beds in Marshall, South Dakota?
What is the number of licensed beds in the state of Missouri?
What is the number of staffed beds in Gasconade in Missouri?
What is the number of licensed beds in Missouri?
What is the number of licensed beds in King William, Virginia?
What is the number of licensed beds in the state of Alabama?
What is the number of ICU beds in Virginia?
What is the number of ICU beds in Mitchell, Texas?
What is the number of ICU beds in Redwood, Minnesota?
What is the number of licensed beds in the state of Illinois?
What is the number of ICU beds in Linn, MO?
What is the numb

In [16]:
 sql_template = 'Select Sum(NUM_Licensed/Staffed/ICU_Beds) From table name Where STATE_Name/COUNTY_Name = input' 
 c_name = 'Maricopa'
 s_name = 'Arizona'
 query = sql_template.replace("NUM_Licensed/Staffed/ICU_Beds", "NUM_STAFFED_BEDS")
 query = query.replace("STATE_Name/COUNTY_Name = input", "COUNTY_NAME = " + c_name)
 query = query + " AND STATE_NAME = " + s_name
print(query)

Select Sum(NUM_STAFFED_BEDS) From table name Where COUNTY_NAME = Maricopa AND STATE_NAME = Arizona


In [25]:
key_list = list(state_dict.keys())
val_list = list(state_dict.values())
print(key_list[val_list.index('Virginia')])


VA
